# 2.1 - MLFlow

### Entranamiento del modelo y subida a MLFlow

In [ ]:
import pandas as pd
import numpy as np

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split as tts

from sklearn.linear_model import ElasticNet


import mlflow
import mlflow.sklearn

In [ ]:
def eval_metrics(actual, pred):
    rmse = np.sqrt(mean_squared_error(actual, pred))
    mae = mean_absolute_error(actual, pred)
    r2 = r2_score(actual, pred)
    return rmse, mae, r2

In [ ]:
data=pd.read_csv('../data/winequality-red.csv', sep=';')

X=data.drop('quality', axis=1)

y=data.quality


X_train, X_test, y_train, y_test=tts(X, y)

In [ ]:
X_test.head()

In [ ]:
data.head()

In [ ]:
y.unique()

In [ ]:
with mlflow.start_run():
    
    elastic=ElasticNet(alpha=0.5, l1_ratio=0.5)
    elastic.fit(X_train, y_train)

    y_pred=elastic.predict(X_test)

    rmse, mae, r2=eval_metrics(y_test, y_pred)

    print('  RMSE: ',  rmse)
    print('  MAE: ',  mae)
    print('  R2: ',  r2)

    mlflow.log_metric('rmse', rmse)
    mlflow.log_metric('r2', r2)
    mlflow.log_metric('mae', mae)

    #registro del modelo
    mlflow.sklearn.log_model(elastic, 'model')
    
    
    
    

### LLamada al servidor de MLFlow

puerto 5000

In [ ]:
!mlflow ui

### Nueva conexion para prediccion por terminal

Puerto 1234

In [ ]:
!mlflow models serve -m mlruns/0/fcc62cbb40c0497c8fe4660b000791d3/artifacts/modelo -p 1234

In [ ]:
!curl -X POST -H "Content-Type:application/json; format=pandas-split" --data '{"columns":["alcohol", "chlorides", "citric acid", "density", "fixed acidity",
"free sulfur dioxide", "pH", "residual sugar", "sulphates", "total sulfur dioxide", "volatile acidity"],
"data":[[12.8, 0.029, 0.48, 0.98, 6.2, 29, 3.33, 1.2, 0.39, 75, 0.66]]}' http://127.0.0.1:1234/invocations


In [ ]:
!curl -X POST -H "Content-Type:application/json; format=pandas-split" --data '{"data":[[12.8, 0.029, 0.48, 0.98, 6.2, 29, 3.33, 1.2, 0.39, 75, 0.66]]}' http://127.0.0.1:1234/invocations

### Prediccion en Python

In [ ]:
modelo='file:///home/iudh/apuntes_clase/semana_8/mlruns/0/dc2f149d432e466f98707baa451d6e8f/artifacts/modelo'

modelo=mlflow.pyfunc.load_model(modelo)


modelo.predict(pd.DataFrame([[0, 1, 0.48, 0, 6.2, 29, 90, 1.2, 45, 75, 0.66]]))

In [ ]:
modelo.predict([[0, 1, 0.48, 0, 6.2, 29, 90, 1.2, 45, 75, 0.66]])